### Notebook 4 - technology graph construciton. Solution 2

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from networkx.readwrite import json_graph
import json
from tqdm import tqdm_notebook as tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
pat = pd.read_csv('data/technology_raw/apat63_99_iso3.csv')
cite = pd.read_csv('data/technology_raw/cite75_99.csv')

In [3]:
# patent dataframe with patent ids as indices
pat_idx = pat.set_index(pat['PATENT'])

In [4]:
for y in tqdm(pd.unique(pat['GYEAR'])[11:]):
    # selected year
    y_pat = pat.loc[pat['GYEAR'] == y].reset_index(drop=True)
    # all citations made by granted patents in this year
    y_cite = cite.loc[np.isin(cite['CITING'].values, y_pat['PATENT'].values)].reset_index(drop=True)
    if y_cite.shape[0] > 0:
        # appending citing_country and cited_country
        ### before that, select cited patents that are present in the "pat" dataframe (filtering step)
        cited_all = y_cite.loc[np.isin(y_cite['CITING'].values, y_pat['PATENT'].values)].reset_index(drop=True)['CITED'].values
        cited_exist = cited_all[np.isin(cited_all, pat_idx.index)]
        y_cite = y_cite.loc[np.isin(y_cite['CITED'].values, cited_exist)]
        ###
        y_cite['citing_country'] = pat_idx.loc[y_cite.loc[np.isin(y_cite['CITING'].values, y_pat['PATENT'].values)]['CITING']]['COUNTRY_ISO3'].values
        y_cite['cited_country'] = pat_idx.loc[cited_exist]['COUNTRY_ISO3'].values
        # determine unique citing countries for this year
        unique_citing = pd.unique(y_cite['citing_country']) 
        # initialize a directed graph
        graph = nx.DiGraph(year=int(y))
        # initialize citation dict to keep track of multiple observations for signle country
        citations_dict = {}
        # iterate through all unique citing countries and calculate sum of observations with other countries
        for citing in unique_citing:
            y_cite_country = y_cite.loc[y_cite['citing_country']==citing]
            for pair_country in pd.unique(y_cite_country['cited_country']):
                y_cite_cc = y_cite_country.loc[y_cite_country['cited_country']==pair_country]
                if y_cite_cc.shape[0] > 0:
                    number_citations = y_cite_cc.shape[0]
                    country_pair = pair_country + '-' + citing
                    if country_pair not in citations_dict.keys():
                        citations_dict[country_pair] = number_citations
                    else:
                        citations_dict[country_pair] += number_citations

        for cc in citations_dict.keys():
            cc_list = cc.split('-')
            country_cited = cc_list[0]
            country_citing = cc_list[1]
            number_citations = citations_dict[cc]
            graph.add_edge(country_cited, country_citing)
            graph[country_cited][country_citing]['Number Citations'] = int(number_citations)

        with open(f'data/preprocessed/technology_graphs_2/technology_graph_{int(y)}.json', 'w') as file:
            json.dump(json_graph.node_link_data(graph), file, indent=4)

        del graph, y_pat, y_cite, cited_all, cited_exist, unique_citing, citations_dict

  0%|          | 0/26 [00:00<?, ?it/s]